In [1]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [2]:
import src.ads_davidjames9610.ads as ads
import src.ads_davidjames9610.useful as useful
import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
from src.classifiers_davidjames9610.test_a.e_config import *
import src.misc_davidjames9610.fe_methods as fe
import src.misc_davidjames9610.proc_methods as pm
from hmmlearn.hmm import GaussianHMM, GMMHMM, BaseHMM
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
import pickle
sls = pickle.load(open('samples_labels.pickle','rb'))
features = pickle.load(open('features.pickle','rb'))
classifiers = pickle.load(open('classifiers.pickle','rb'))

In [4]:
CLASSIFICATION = 'classification'
REGRESSION = 'regression'

eval_type = CLASSIFICATION

In [6]:
import d_classifiers_to_results
reload(d_classifiers_to_results)

classification_results = d_classifiers_to_results.get_classification_results(
    features, classifiers, sls, True)


testing for feature type:  lp


KeyboardInterrupt: 

In [ ]:
# combined hmm approach, still Viterbi, this assumes I have longish real input data,
import src.misc_davidjames9610.decode_combine as dc
reload(dc)

# decode_combine_results = d_classifiers_to_results.decode_combine_results()

results = {}  # one for each process method

# CLASSIFICATION
for feature_key in features:

    print('testing for feature type: ', feature_key)
    cv_index = 0    # todo think about updating this cv index for testing
    curr_features = features[feature_key]['val_features'][cv_index]
    curr_label = features[feature_key]['val_label'][cv_index]
    curr_classifiers = classifiers[feature_key]

    combined_model = dc.DecodeCombineGaussian([hmm for hmm in curr_classifiers])

    _, val_pred, val_log_prob = combined_model.decode_hmmlearn(np.concatenate(curr_features))
